<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-700)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [3]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


array(['ACC', 'ADANIENT', 'ADANIGREEN', 'ADANIPORTS', 'ADANITRANS',
       'AMBUJACEM', 'APOLLOHOSP', 'ASIANPAINT', 'AUBANK', 'AUROPHARMA',
       'AXISBANK', 'BAJAJ-AUTO', 'BAJAJFINSV', 'BAJAJHLDNG', 'BAJFINANCE',
       'BANDHANBNK', 'BANKBARODA', 'BATAINDIA', 'BERGEPAINT',
       'BHARTIARTL', 'BIOCON', 'BOSCHLTD', 'BPCL', 'BRITANNIA', 'CANBK',
       'CHOLAFIN', 'CIPLA', 'COALINDIA', 'COLPAL', 'DABUR', 'DEEPAKNTR',
       'DIVISLAB', 'DLF', 'DMART', 'DRREDDY', 'EICHERMOT', 'EMAMILTD',
       'FEDERALBNK', 'GAIL', 'GILLETTE', 'GLAND', 'GODREJCP', 'GODREJIND',
       'GRASIM', 'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCAMC', 'HDFCBANK',
       'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO', 'HINDPETRO', 'HINDUNILVR',
       'IBREALEST', 'ICICIBANK', 'ICICIGI', 'ICICIPRULI', 'IGL',
       'INDIANB', 'INDIGO', 'INDIGOPNTS', 'INDUSINDBK', 'INDUSTOWER',
       'INFY', 'IOC', 'IRCTC', 'ITC', 'JINDALSTEL', 'JSWSTEEL',
       'JUBLFOOD', 'KOTAKBANK', 'LT', 'LTIM', 'LUPIN', 'M&M', 'MARICO',
       'MARUTI', '

In [4]:
# star stocks & my stocks
star_stocks = list(mypf[mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(mypf[mypf['InPortfolio'] == 1]['Symbol'].values)

In [5]:
# update Nifty features
'''
stock = 'INFY'
column = 'Conviction'
value = 'H'
nifty100_df.loc[nifty100_df['Symbol'] == stock, column] = value

mypf.to_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv',index=False)
'''
mypf['Conviction'].value_counts()

L    67
H    31
M    27
Name: Conviction, dtype: int64

In [6]:
# star stocks
mypf['StarStock'].value_counts()

0    70
1    55
Name: StarStock, dtype: int64

In [7]:
# latest quarter
mypf['LatestQ'].value_counts()

0    77
1    48
Name: LatestQ, dtype: int64

In [8]:
# stocks in portfolio
mypf['InPortfolio'].value_counts()

0    110
1     15
Name: InPortfolio, dtype: int64

In [9]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [10]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()  
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [11]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 150
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='3y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_150'] = round((stock_df['Close'] - stock_df['150_SMA'])*100/stock_df['150_SMA'],2)
    stock_df['150_SMA_Lag10'] = stock_df['150_SMA'].shift(10)
    stock_df['Slope%_150'] = round((stock_df['150_SMA'] - stock_df['150_SMA_Lag10'])*100/stock_df['150_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    
    stock_df.ta.rsi(append=True)
    stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0)    
    stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)   
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Upside%ATH'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','150_SMA','150_SMA_Lag10','50_SMA_Lag5'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    if stock_id in my_stocks:
      current_pe = get_current_pe(stock_id)
    else:
      current_pe = 1000

    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]    
    tmp['Curr_PE'] = current_pe
    tmp['EstmtFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['EstmtFlag'].values[0] 
    tmp['DuraFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['DuraFlag'].values[0]   
    tmp['Conviction'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]    
    tmp['LatestQ'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQ'].values[0]      
    df_prec_dev = df_prec_dev.append(tmp)

In [12]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,,
2023-04-05 00:00:00+05:30,17557.0,^NSEI,-1.35,-0.20,-0.44,58.0,18812.0,15294.0,35.67,7.0
2023-04-13 00:00:00+05:30,17828.0,^NSEI,0.14,-0.02,-0.13,66.0,18812.0,15294.0,27.97,6.0
2023-04-21 00:00:00+05:30,17624.0,^NSEI,-0.98,0.01,-0.02,56.0,18812.0,15294.0,33.77,7.0
2023-04-28 00:00:00+05:30,18065.0,^NSEI,1.49,-0.02,0.02,71.0,18812.0,15294.0,21.23,4.0
2023-05-08 00:00:00+05:30,18278.0,^NSEI,2.56,0.13,0.13,69.0,18812.0,15294.0,15.18,3.0


In [13]:
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

mypf['Stock'] = mypf['Symbol'] + '.NS'
#df_prec_dev.drop(['Med_PE'], axis=1, inplace=True)

In [14]:
df_prec_dev['Discount%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Med_PE']),0)
#df_prec_dev['Upside%Estimate'] = round((df_prec_dev['Max']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
df_prec_dev = df_prec_dev.set_index('Stock')

In [15]:
# portfolio stocks
cols = ['Close',	'Dev%_150',	'Slope%_150',	'Slope%_50',	'RSI',	'Discount%',	'Upside%ATH',	'Avg_Cost',	'Med_PE',	'Curr_PE',	'Conviction', 'Discount%PE', 'LatestQ']
tmp_df = df_prec_dev[cols]
tmp_df = tmp_df[tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 15


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQ
Stock,,,,,,,,,,,,,
INDIGOPNTS.NS,1268.0,1.77,-2.09,1.03,81.0,83.19,111.0,1152.30,104.9,51.2,H,51.0,0
JUBLFOOD.NS,471.0,-7.01,-2.17,0.00,65.0,89.81,92.0,448.52,90.5,73.8,H,18.0,0
IRCTC.NS,628.0,-4.52,-0.76,-0.17,66.0,69.97,85.0,618.80,77.4,53.4,H,31.0,0
INFY.NS,1268.0,-14.52,-0.61,-2.30,39.0,93.33,50.0,1252.12,29.0,22.0,H,24.0,1
BATAINDIA.NS,1494.0,-6.30,-1.63,0.14,63.0,86.39,45.0,1495.89,126.0,59.9,H,52.0,0
DMART.NS,3672.0,-3.32,-1.53,0.20,71.0,76.91,45.0,3501.37,160.1,102.0,H,36.0,0
PAGEIND.NS,41250.0,-4.01,-1.44,0.59,68.0,50.35,30.0,38201.00,97.7,67.4,H,31.0,0
GILLETTE.NS,4596.0,-4.84,-1.10,-0.56,64.0,76.68,29.0,4353.00,61.2,50.3,M,18.0,0
PIDILITIND.NS,2434.0,-2.57,-1.08,0.55,57.0,51.37,19.0,2442.09,94.6,99.5,H,-5.0,0


In [16]:
df_prec_dev.drop([ 'Avg_Cost', 'Shares', 'Med_PE', 'Curr_PE', 'Discount%PE'], axis=1, inplace=True)

In [17]:
# all time high sales and profit TTM

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 9


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
ADANIENT.NS,1901.0,-33.80,-4.18,0.79,55.0,4165.0,1194.0,76.20,119.0,1.0,0.0,M,1
LTIM.NS,4520.0,-1.14,-0.09,-0.54,53.0,7439.0,3761.0,79.36,65.0,1.0,0.0,M,1
ICICIGI.NS,1085.0,-5.32,-0.79,-0.46,51.0,1620.0,1060.0,95.54,49.0,1.0,0.0,M,1
BANDHANBNK.NS,234.0,-0.40,-1.70,0.00,67.0,346.0,187.0,70.44,48.0,1.0,1.0,L,0
TATAPOWER.NS,203.0,-3.40,-0.95,0.00,58.0,287.0,111.0,47.73,41.0,1.0,0.0,M,1
BERGEPAINT.NS,619.0,4.93,-0.34,1.02,67.0,858.0,540.0,75.16,39.0,1.0,0.0,M,0
SBICARD.NS,815.0,3.78,-1.27,0.53,72.0,1133.0,673.0,69.13,39.0,1.0,0.0,M,1
CIPLA.NS,926.0,-10.15,-0.97,-1.11,57.0,1173.0,856.0,77.92,27.0,1.0,0.0,M,0
TCS.NS,3248.0,0.20,0.46,-0.90,56.0,3886.0,2915.0,65.71,20.0,1.0,0.0,M,1


In [18]:
# all time high sales and profit TTM

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=True)

qualified stocks: 17


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
LT.NS,2375.0,12.24,1.32,0.81,68.0,2384.0,1432.0,0.95,0.0,1.0,0.0,M,0
ITC.NS,430.0,20.59,1.68,1.27,80.0,430.0,189.0,0.00,0.0,1.0,0.0,M,0
TITAN.NS,2753.0,8.22,0.00,0.72,77.0,2770.0,1662.0,1.53,1.0,1.0,1.0,H,1
BRITANNIA.NS,4580.0,9.23,1.31,0.16,68.0,4626.0,2945.0,2.74,1.0,1.0,0.0,M,1
ICICIBANK.NS,941.0,6.34,0.23,0.69,76.0,953.0,619.0,3.59,1.0,1.0,1.0,H,1
INDIANB.NS,323.0,17.77,2.92,1.72,60.0,334.0,112.0,4.95,3.0,1.0,1.0,H,0
AUBANK.NS,691.0,10.75,0.00,0.97,65.0,717.0,493.0,11.61,4.0,1.0,0.0,M,1
HDFC.NS,2743.0,5.17,0.92,0.30,52.0,2963.0,2053.0,24.18,8.0,1.0,0.0,M,1
SBIN.NS,581.0,2.83,0.00,0.74,68.0,626.0,400.0,19.91,8.0,1.0,0.0,M,0


In [19]:
# stocks excluded from all time high

excluded_stocks = ['ACC.NS','AMBUJACEM.NS','CADILAHC.NS','GILLETTE.NS','NESTLEIND.NS','PGHH.NS','SIEMENS.NS']

tmp_df = df_prec_dev[df_prec_dev.index.isin(excluded_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 5


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
ACC.NS,1762.0,-17.93,-2.61,-0.46,52.0,2748.0,1614.0,86.95,56.0,0.0,0.0,L,0
AMBUJACEM.NS,407.0,-12.38,-2.16,1.07,63.0,588.0,275.0,57.83,44.0,0.0,0.0,L,0
PGHH.NS,13871.0,0.11,-0.22,0.20,47.0,15778.0,12000.0,50.48,14.0,0.0,0.0,L,0
NESTLEIND.NS,21925.0,12.23,1.00,1.41,77.0,22020.0,15919.0,1.56,0.0,1.0,1.0,H,0
SIEMENS.NS,3683.0,21.51,1.22,1.23,81.0,3683.0,1922.0,0.00,0.0,1.0,1.0,H,0


In [20]:
# prospects with heavy upside
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 2


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
INDUSTOWER.NS,151.0,-13.39,-2.3,-1.32,56.0,303.0,137.0,91.57,101.0,1.0,1.0,H,1
BANDHANBNK.NS,234.0,-0.40,-1.7,0.00,67.0,346.0,187.0,70.44,48.0,1.0,1.0,L,0


In [21]:
# prospects near 52W low
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] > 60]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=False)

qualified stocks: 3


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
INDUSTOWER.NS,151.0,-13.39,-2.30,-1.32,56.0,303.0,137.0,91.57,101.0,1.0,1.0,H,1
HDFCLIFE.NS,548.0,1.93,-0.37,0.59,60.0,757.0,461.0,70.61,38.0,1.0,1.0,H,1
BANDHANBNK.NS,234.0,-0.40,-1.70,0.00,67.0,346.0,187.0,70.44,48.0,1.0,1.0,L,0


In [22]:
# prospects near 52W high
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 7


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
SIEMENS.NS,3683.0,21.51,1.22,1.23,81.0,3683.0,1922.0,0.00,0.0,1.0,1.0,H,0
TITAN.NS,2753.0,8.22,0.00,0.72,77.0,2770.0,1662.0,1.53,1.0,1.0,1.0,H,1
NESTLEIND.NS,21925.0,12.23,1.00,1.41,77.0,22020.0,15919.0,1.56,0.0,1.0,1.0,H,0
ICICIBANK.NS,941.0,6.34,0.23,0.69,76.0,953.0,619.0,3.59,1.0,1.0,1.0,H,1
INDIANB.NS,323.0,17.77,2.92,1.72,60.0,334.0,112.0,4.95,3.0,1.0,1.0,H,0
BANKBARODA.NS,180.0,8.73,1.81,1.18,56.0,193.0,71.0,10.66,7.0,1.0,1.0,H,0
MARUTI.NS,9107.0,4.88,-0.36,0.08,74.0,9528.0,6565.0,14.21,5.0,1.0,1.0,H,1


In [23]:
# prospects in bull run
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_150'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 3


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
NESTLEIND.NS,21925.0,12.23,1.00,1.41,77.0,22020.0,15919.0,1.56,0.0,1.0,1.0,H,0
SIEMENS.NS,3683.0,21.51,1.22,1.23,81.0,3683.0,1922.0,0.00,0.0,1.0,1.0,H,0
ICICIBANK.NS,941.0,6.34,0.23,0.69,76.0,953.0,619.0,3.59,1.0,1.0,1.0,H,1


In [24]:
# prospects about to turn around
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_150'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 5


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
HDFCLIFE.NS,548.0,1.93,-0.37,0.59,60.0,757.0,461.0,70.61,38.0,1.0,1.0,H,1
MARUTI.NS,9107.0,4.88,-0.36,0.08,74.0,9528.0,6565.0,14.21,5.0,1.0,1.0,H,1
BANDHANBNK.NS,234.0,-0.40,-1.70,0.00,67.0,346.0,187.0,70.44,48.0,1.0,1.0,L,0
INDUSINDBK.NS,1119.0,-2.44,-0.52,-0.18,52.0,1264.0,771.0,29.41,13.0,1.0,1.0,H,1
INDUSTOWER.NS,151.0,-13.39,-2.30,-1.32,56.0,303.0,137.0,91.57,101.0,1.0,1.0,H,1


In [25]:
# cross-check the features
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] != 1) | (tmp_df['DuraFlag'] != 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 40


,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
ADANITRANS.NS,949.0,-54.70,-9.50,-0.32,42.0,4105.0,643.0,91.16,333.0,1.0,-1.0,L,0
PEL.NS,744.0,-7.19,-1.87,-1.80,59.0,2866.0,635.0,95.11,285.0,-1.0,0.0,L,1
ADANIGREEN.NS,918.0,-39.07,-6.24,3.73,51.0,2970.0,462.0,81.82,224.0,-1.0,-1.0,L,1
GLAND.NS,1401.0,-10.06,-3.72,0.39,63.0,4316.0,1154.0,92.19,208.0,1.0,0.0,M,0
IBREALEST.NS,69.0,-2.32,-1.41,1.69,63.0,191.0,47.0,84.72,177.0,-1.0,-1.0,L,0
ADANIENT.NS,1901.0,-33.80,-4.18,0.79,55.0,4165.0,1194.0,76.20,119.0,1.0,0.0,M,1
NETWORK18.NS,57.0,-7.74,-1.61,0.00,52.0,114.0,45.0,82.61,100.0,-1.0,-1.0,L,1
ZEEL.NS,191.0,-17.90,-2.15,-0.99,39.0,367.0,164.0,86.70,92.0,-1.0,1.0,L,0
WIPRO.NS,380.0,-2.21,-0.51,-0.80,54.0,713.0,356.0,93.28,88.0,-1.0,0.0,L,1


In [26]:
stock_id = 'CHOLAFIN.NS'
df_prec_dev[df_prec_dev.index == stock_id]

,Close,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction,LatestQ
Stock,,,,,,,,,,,,,
CHOLAFIN.NS,1000.0,33.16,1.07,2.0,87.0,1000.0,473.0,0.0,0.0,NaN,NaN,L,1
